In [1]:
import wave
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

import scipy
from scipy import integrate
import scipy.optimize
import astropy
import astropy.io as io
import astropy.nddata
import astropy.constants as const
import astropy.units as units
import specutils
from tqdm import tqdm

import inspect

import FunctionLib as FL

In [ ]:

fitter.iterative_gaussian_fitting(4863.0 * units.Angstrom, tolerance=10 * units.AA, plot_results=True)

In [ ]:
fitter.fit_results